## EQLock: Statistics of Earthquakes and Detector Lockloss

**Introduction:**
insert short intro

**Input:**
* Parameters: start time, duration, width, prominience, distance
* Channels: choose frequency band and ETM/ITM channels of interest

**Output:**
Probability

In [ ]:
import pandas as pd
import numpy as np

from gwpy.timeseries import TimeSeries
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
from scipy.signal import find_peaks
import datetime
from gwpy.time import tconvert
from gwpy.time import from_gps
from numpy import sqrt
import math
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last"
%matplotlib inline

### Section 1: set parameters & get data
* This is most time-consuming part of code! For one month of data, it typically takes between 2-5 hours to get the seismic TimeSeries and 10-30 minutes to get lockbit TimeSeries.
* If importing large amounts of data, I recommend breaking up the time and repeating Sections 1 and 2. If broken up monthly, for example, Section 1 saves two csv files for each month. Section 2 will be explained.

In [ ]:
#parameters
start_time = 1251331218 #Sep 1, 2019 00:00:00
duration = 24*3600*30 #30 days in Sep #can also use GPS (end time - start time)

#vertical + horizontal channels for end stations ETMX, ETMY and inner station ITMY
channels = ['ETMX_Z', 'ETMY_Z', 'ITMY_Z', 'ETMX_X','ETMX_Y', 'ETMY_X', 'ETMY_Y', 'ITMY_X', 'ITMY_Y']

In [ ]:
%%time #note: getting data takes longest, so time this cell to understand efficiency
#get time series in earthquake band ground motion

channelRMS = []
for chn in channels:
    chname = 'L1:ISI-GND_STS_' + chn + '_BLRMS_30M_100M.max' #earthquake band 0.03-0.1 Hz
    data = TimeSeries.find(chname, start_time, start_time + duration, frametype ='L1_M', verbose=True) #L1_M is minute-trend
    channelRMS.append(data.value)

In [ ]:
#store earthquake data
eqdata = pd.DataFrame(np.transpose(channelRMS), columns = [channels])

In [ ]:
#save to csv #keep date, frequency, and location in file name
eqdata.to_csv("eqdata_sep2019_30M_100M_LA.csv")

In [ ]:
%%time
#find time series for lockbit data

lockstate = []
lockbit_ts = TimeSeries.find('L1:GRD-ISC_LOCK_STATE_N.min',start_time, start_time + duration, frametype ='L1_M', verbose=True)
lockstate.append(lockbit_ts.value)

In [ ]:
#store lockbit data
lbdata = pd.DataFrame(np.transpose(lockstate)) 

In [ ]:
#save to csv
lbdata.to_csv("lbdata_sep2019_LA.csv") 

### Section 2: combine data & get statistics
* The three parameters set here define what is considered a peak in the data, thus what characterizes an earthquake.
* 

In [ ]:
#set parameters
peakwidth = 5
peakprom = 100
peakdist = 120

In [ ]:
#get earthquake dataframe from file (in folder eqlockbit_data)
lockbitdf = pd.read_csv("./eqlockbit_data/lbdata_sep2019_LA.csv")
eqdatadf = pd.read_csv("./eqlockbit_data/eqdata_sep2019_30M_100M_LA.csv")

etmx = sqrt((eqdatadf['ETMX_X'])**2 + (eqdatadf['ETMX_Y'])**2)
etmy = sqrt((eqdatadf['ETMY_X'])**2 + (eqdatadf['ETMY_Y'])**2)
itmy = sqrt((eqdatadf['ITMY_X'])**2 + (eqdatadf['ITMY_Y'])**2)

eqdatadf['ETMX_H'] = etmx
eqdatadf['ETMY_H'] = etmy
eqdatadf['ITMY_H'] = itmy

In [ ]:
#find peaks using parameters

peaks,_ = find_peaks(eqdatadf['ETMX_Z'], width = peakwidth, distance = peakdist, prominence = peakprom)
peaks

In [ ]:
#find common peaks in at least two of the three Z-channels, define those as EQs
EQix = []

EQix.append(np.intersect1d(peaksRMSix[0],peaksRMSix[1]))
EQix.append(np.intersect1d(peaksRMSix[1],peaksRMSix[2]))
EQix.append(np.intersect1d(peaksRMSix[2],peaksRMSix[0]))

EQix = np.unique(np.concatenate(EQix))
EQix #returns list of location of each earthquake peak

In [ ]:
#plot common peaks in ETMX_Z, ETMY_Z, and ITMY_Z
plt.figure(figsize=(12,8))
plt.title('L1: Common peaks in vertical motion channels',fontsize=18)

colors = ['blue','green','purple']
for jj in range(3):
    plt.plot(eqdatadf[channels[jj]], color = colors[jj])

for jj in range(3):
    for i in range(len(EQix)):
        plt.plot(EQix[i], eqdatadf[channels[jj]][EQix[i]], '*', color = 'orange')

plt.grid(True, which="both")
plt.legend(['ETMX_Z', 'ETMY_Z', 'ITMY_Z', 'Earthquakes'])
labels = ['09-01', '09-08', '09-15', '09-22', '09-29'] #labels for each week
plt.xticks(np.arange(min(t), max(t)+1, step=10080), labels)
plt.xlabel('Time from 2019-09-01 00:00:00 UTC', fontsize=15)
plt.ylabel('Ground motion [nm/s]',fontsize=15)
plt.yscale('log')
plt.show()

In [ ]:
#find if detector survived EQ or lost lock status
survived = []
lostlock = []
delta = 20  #minutes - interval before and after EQ to define survival

for jj in EQix:
    if np.min(lockbitdf['0'][jj-delta:jj+delta])>=2000:
        survived.append(jj)
    else:
        lostlock.append(jj)

In [ ]:
#find times of lockloss within +/- delta of each event 
#outputs first number we lost lock
lostlockbit = []

for jj in lostlock:
    d0 = delta
    while lockbitdf['0'][jj-d0]>=2000:
        d0 = d0 - 1    
    lostlockbit.append(jj-d0)  #number we want to associate w eq

In [ ]:
#for survived & lost lock, find max vertical and max horizontal
#vertical = 'ETMX_Z', 'ETMY_Z', 'ITMY_Z'
#horizontal = 'ETMX_X','ETMX_Y', 'ETMY_X', 'ETMY_Y', 'ITMY_X', 'ITMY_Y'

eqdata_total = []

EQix_secs = [element * 60 for element in EQix]
EQix_gps = [element + start_time for element in EQix_secs]
lostlock_secs = [element * 60 for element in lostlockbit]
lostlock_gps = [element + start_time for element in lostlock_secs] #time when first lost lock

for i in range(len(EQix)):
    date = from_gps(EQix_gps[i]) #date of earthquake
    tpeak = EQix_gps[i]
    
    #vertical motion at peak
    maxv_v = max(eqdatadf['ETMX_Z'][EQix[i]], eqdatadf['ETMY_Z'][EQix[i]], eqdatadf['ITMY_Z'][EQix[i]])
    #horizontal motion at peak
    etmxh = sqrt((eqdatadf['ETMX_X'][EQix[i]])**2 + (eqdatadf['ETMX_Y'][EQix[i]])**2)
    etmyh = sqrt((eqdatadf['ETMY_X'][EQix[i]])**2 + (eqdatadf['ETMY_Y'][EQix[i]])**2)
    itmyh = sqrt((eqdatadf['ITMY_X'][EQix[i]])**2 + (eqdatadf['ITMY_Y'][EQix[i]])**2)
    maxv_h = max(etmxh, etmyh, itmyh)
    
    ratio = maxv_v/maxv_h

    if pd.Series(EQix[i]).isin(survived).any():
        brlock = "No"  #did earthquake break the lock?
        tlostv = "-"
        lostv_v = "-"
        lostv_h = "-"
    if pd.Series(EQix[i]).isin(lostlock).any():
        brlock = "Yes"
        j=0
        while EQix[i] != lostlock[j]:
            j=j+1
        tlostv = lostlock_gps[j]
        
        #vertical motion when lost lock
        lostv_v = max(eqdatadf['ETMX_Z'][lostlockbit[j]], eqdatadf['ETMY_Z'][lostlockbit[j]], eqdatadf['ITMY_Z'][lostlockbit[j]])
        #horizontal motion when lost lock
        etmxh_ll = sqrt((eqdatadf['ETMX_X'][lostlockbit[j]])**2 + (eqdatadf['ETMX_Y'][lostlockbit[j]])**2)
        etmyh_ll = sqrt((eqdatadf['ETMY_X'][lostlockbit[j]])**2 + (eqdatadf['ETMY_Y'][lostlockbit[j]])**2)
        itmyh_ll = sqrt((eqdatadf['ITMY_X'][lostlockbit[j]])**2 + (eqdatadf['ITMY_Y'][lostlockbit[j]])**2)
        lostv_h = max(etmxh_ll, etmyh_ll, itmyh_ll)
    
    eqdata_total.append([date, brlock, tpeak, maxv_v, maxv_h, ratio, tlostv, lostv_v, lostv_h])

In [ ]:
#store dataframe
eqtotaldf = pd.DataFrame(eqdata_total, columns = ['Date', 'Broke Lock', 'Time of Peak', 'Vert. Vel.', 'Horiz. Vel.', 'Ratio V/H', 'Time of Lost Lock (LL)', 'LL Vert. Vel.', 'LL Horiz. Vel.'])
eqtotaldf

In [ ]:
#save to csv
eqtotaldf.to_csv("eqtotaldf_sep2019_30M_100M_LA.csv") 

### Section 3:
* Only necessary to concatenate csv files if more than one is saved.


In [ ]:
#get earthquake total dataframes from file (in folder eqlock)
apr = pd.read_csv("./eqtotaldf_apr2019_30M_100M_LA.csv")
may = pd.read_csv("./eqtotaldf_may2019_30M_100M_LA.csv")
jun = pd.read_csv("./eqtotaldf_jun2019_30M_100M_LA.csv")
jul = pd.read_csv("./eqtotaldf_jul2019_30M_100M_LA.csv")
aug = pd.read_csv("./eqtotaldf_aug2019_30M_100M_LA.csv")
sep = pd.read_csv("./eqtotaldf_sep2019_30M_100M_LA.csv")

nov = pd.concat(map(pd.read_csv, ["./eqtotaldf_novA2019_30M_100M_LA.csv", "./eqtotaldf_novB2019_30M_100M_LA.csv"]), ignore_index=True)
dec = pd.concat(map(pd.read_csv, ["./eqtotaldf_decA2019_30M_100M_LA.csv", "./eqtotaldf_decB2019_30M_100M_LA.csv"]), ignore_index=True)
jan = pd.read_csv("./eqtotaldf_jan2020_30M_100M_LA.csv")
feb = pd.read_csv("./eqtotaldf_feb2020_30M_100M_LA.csv")
mar = pd.read_csv("./eqtotaldf_mar2020_30M_100M_LA.csv")

In [ ]:
months = [apr, may, jun, jul, aug, sep, nov, dec, jan, feb, mar]
o3ab_df = pd.concat(months)

In [ ]:
o3ab_df.to_csv("eqtotaldf_o3ab_30M_100M_LA.csv")

In [ ]:
totaldf = pd.read_csv("eqtotaldf_o3ab_30M_100M_LA.csv")

In [ ]:
plt.figure(figsize=(12,8))
plt.title('L1: Peak ground velocities of earthquake events (O3a and O3b)', fontsize=18)

plt.scatter(totaldf[totaldf['Broke Lock']=='No']['Horiz. Vel.'], totaldf[totaldf['Broke Lock']=='No']['Vert. Vel.'],label='Survived', color='darkblue')
plt.scatter(totaldf[totaldf['Broke Lock']=='Yes']['Horiz. Vel.'], totaldf[totaldf['Broke Lock']=='Yes']['Vert. Vel.'],label='Broke Lock', color='turquoise')
plt.plot([100,100000], [100, 100000], '-')

plt.xlabel('Horizontal Peak Velocity (nm/s)', fontsize=15)
plt.ylabel('Vertical Peak Velocity (nm/s)',fontsize=15)
plt.legend()
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which="both", ls="-")

plt.show()

In [ ]:
bins = np.logspace(2, 5.9, num=14)

In [ ]:
survivedeq = totaldf[totaldf['Broke Lock']=='No']['Vert. Vel.'].value_counts(sort = False, bins = bins)
totaleq = totaldf['Vert. Vel.'].value_counts(sort = False, bins = bins)

prob = survivedeq/totaleq

In [ ]:
# plot rectangles instead of points (histogram)
dimw = np.diff(bins)

fig, ax = plt.subplots(figsize=(12,8))
x = bins[0:-1]
y = prob
b = ax.bar(x + dimw, y+0.01, dimw, bottom=-0.001, yerr=yerr, ls='none')

#ax.set_xticks(x + dimw / 2, labels=map(str, x))
ax.set_xscale('log')
ax.set_ylim(0, 1)
ax.set_xlabel('Vertical ground motion (nm/s)')
ax.set_ylabel('Probability of Survival')

plt.show()